## Database Engineer Capstone Week 3

***

### Import Libraries

In [5]:
!pip3 install pandas    
!pip3 install numpy
!pip3 install matplotlib
!pip3 install seaborn
!pip3 install scipy
!pip3 install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 997.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 7.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 9.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 kB 9.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 7.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.7/163.7 kB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 4.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 kB 5.0 MB/s eta 0:00:00a 0

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

import datetime
from datetime import datetime, timedelta
import scipy.stats


import mysql.connector as connector

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
#sets the default autosave frequency in seconds
%autosave 60
sns.set_style('dark')
sns.set(font_scale=1.2)

plt.rc('axes', titlesize=9)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.option_context('float_format','{:.2f}'.format)

random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

Matplotlib is building the font cache; this may take a moment.


Autosaving every 60 seconds


## Load Data from SQL database

Your first step is to import the connector module, enter your user details and connect with the database (Hint: you can use an alias when importing the module).

### MySQL

In [87]:
#Create a connection

mydb = connector.connect(
  user="root",
  passwd="root",
  database="LittleLemonDB"
)

In [63]:
print(mydb)

In [88]:
mycursor = mydb.cursor()

In [10]:
mycursor

In [89]:
mydb.reconnect() #Reconnect cursor

In this second task, you now need to query the database to show all tables within the database.

In [72]:
pd.read_sql_query('SHOW tables', mydb)

,Tables_in_LittleLemonDB
0,Booking
1,Customer
2,DeliveryStatus
3,Menu
4,Orders
5,Staff


In [14]:
pd.read_sql_query("SHOW columns FROM Booking", mydb)

,Field,Type,Null,Key,Default,Extra
0,BookingID,int,NO,PRI,None,
1,Date,datetime,NO,,None,
2,TableNumber,int,NO,,None,


An alternate way to learn the same information would be to use the DESCRIBE function. The syntax is:

In [15]:
pd.read_sql_query("DESCRIBE Booking", mydb)

,Field,Type,Null,Key,Default,Extra
0,BookingID,int,NO,PRI,None,
1,Date,datetime,NO,,None,
2,TableNumber,int,NO,,None,


For the third and final task, Little Lemon need you to return specific details from your database. They require the full name and contact details for every customer that has placed an order greater than $60 for a promotional campaign.

In [16]:
pd.read_sql_query("SELECT * FROM Customer", mydb)

,CustomerID,FullName,ContactNumber,Email
0,1,Alice Johnson,222-333-4444,alice.johnson@example.com
1,2,Bob Harris,777-888-9999,bob.harris@example.com
2,3,Charlie Williams,555-666-7777,charlie.williams@example.com


In [66]:
pd.read_sql_query("SELECT * FROM Orders", mydb)

,OrderID,Date,Quantity,TotalCost,Booking_BookingID,Customer_CustomerID,Menu_MenuID,DeliveryStatus_DeliveryID
0,1,2024-10-28 18:30:00,2,40.0,1,1,1,1
1,2,2024-10-29 19:30:00,1,20.0,2,2,2,2
2,3,2024-10-30 20:30:00,5,60.0,3,3,3,3


In [18]:
pd.read_sql_query("""SELECT Customer.FullName, Customer.ContactNumber FROM Customer
                    INNER JOIN Orders
                    ON Customer.CustomerID = Orders.Customer_CustomerID
                    WHERE Orders.TotalCost > 60
                    """, mydb)

,FullName,ContactNumber


The cloned repository contains a procedure called GetMaxQuantity(). Call this procedure and verify

In [67]:
pd.read_sql_query("""CALL GetMaxQuantity();""", mydb)

,MAX(Orders.Quantity)
0,5


Call the ManageBooking() procedure by passing the appropriate parameters. First with an available table number, then with one that has already been reserved.

In [56]:
pd.read_sql_query("""CALL ManageBooking("2022-01-01", 5);""", mydb)

DatabaseError: Execution failed on sql 'CALL ManageBooking("2022-01-01", 5);': 1305 (42000): PROCEDURE LittleLemonDB.ManageBooking does not exist

Create an SQL statement that calls the AddBooking() procedure

Call the cursor .execute() method using the above parameters. Print out the result using the cursor .fetchall() method.

Create a SQL statement that calls the UpdateBooking() procedure

In [59]:
mycursor.execute("""CALL AddBooking(99,99,99,"2022-12-10");""")

In [46]:
mycursor.fetchall()

[]

Create a SQL statement that calls the CancelBooking() procedure

In [49]:
mycursor.execute("""CALL UpdateBooking(99,"2022-01-10");""")

In [50]:
mycursor.fetchall()

[]

In [90]:
mycursor.execute("""CALL CancelBooking(99);""")